# PyMuPDF

References: https://towardsdatascience.com/extracting-headers-and-paragraphs-from-pdf-using-pymupdf-676e8421c467

Pros:
Light-weight approach

Cons:
Since this is a rule-based approach, therea are some document not covered by the rules.

In [26]:
!pip install PyMuPDF

     |████████████████████████████████| 6.4 MB 1.1 MB/s eta 0:00:01


In [32]:
import fitz
from operator import itemgetter

In [30]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [36]:
# Note, we’re only considering the font-sizes here, 
# but with a few extra lines of code, 
# you can find a way to incorporate the other attributes 
# if you’re using the granularity=True flag in the fonts() function!

def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [45]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [86]:
doc = fitz.open("data/essay_2.pdf")

In [87]:
detected_fonts_styles = fonts(doc)
detected_fonts_styles

([('11.039999961853027', 75), ('11.520000457763672', 31), ('12.0', 1)],
 {'11.039999961853027': {'size': 11.039999961853027, 'font': 'Calibri'},
  '12.0': {'size': 12.0, 'font': 'TimesNewRomanPSMT'},
  '11.520000457763672': {'size': 11.520000457763672, 'font': 'Calibri'}})

In [88]:
size_tag = font_tags(detected_fonts_styles[0],detected_fonts_styles[1])
size_tag

{12.0: '<h1>', 11.520000457763672: '<h2>', 11.039999961853027: '<p>'}

In [89]:
tagged_list = headers_para(doc, size_tag)
tagged_list

['<p>Pemerintah Indonesia ingin menurunkan emisi sebesar 29% pada tahun 2030 sehubungan dengan | partisipasi Indonesia dalam Perjanjian Paris 2015 [1]. Tindakan iklim menjadi fokus global, urgensi untuk | mengatasi semua emisi karbon dioksida dari banyak negara di seluruh dunia. Perhatian pemerintah | pertama-tama difokuskan pada sektor ketenagalistrikan untuk memanfaatkan energi terbarukan untuk | mengurangi emisi global. Dengan meningkatkan energi terbarukan, negara dapat secara tajam | mengurangi satu sumber utama terkait emisi karbon dioksida. Indonesia menetapkan target energi | terbarukan dalam bauran energi sebesar 23% pada tahun 2025, pada Q3 tahun 2020 persentasenya hanya | 10,9% [2,3]. Kesenjangan untuk mencapai tujuan sekitar 14,1%, itu harus melipatgandakan usaha empat | kali dari bisnis seperti biasa [3]. |',
 '|',
 '<p>Menurut konteks global energi trilema, biasanya digunakan untuk menyeimbangkan keamanan energi, | ekuitas energi, dan keberlanjutan energi. Indonesia membu

In [90]:
doc = fitz.open("data/essay_1.pdf")
detected_fonts_styles = fonts(doc)
size_tag = font_tags(detected_fonts_styles[0],detected_fonts_styles[1])
tagged_list = headers_para(doc, size_tag)
tagged_list

['<h1>Agrophovoltaic (APV)  |',
 '',
 '<p>1. Pendahuluan |',
 '<p>Musim kemarau di Indonesia menyebabkan angka produksi padi menurun, penurunan |',
 '<p>sebesar 4,6 juta ton terjadi di tahun 2019 [1].  Gambar 1 menunjukan luas fase persiapan |',
 '<p>lahan terjadi penurunan pada bulan April – Agustus yang sesuai dengan musim kemarau di |',
 '<p>Indonesia. Provinsi yang memiliki luas fase persiapan terbesar diantaranya Jawa Barat, |',
 '<p>Jawa Tengah, Jawa Timur, Sulawesi Selatan, dan Sumatera Selatan.  |',
 '|',
 '<p>Gambar 1. Luas fase persiapan lahan 2018 – 2019 (Dimodifikasi dari BPS, 2019) |',
 '<p>Selain itu, provinsi tersebut memiliki tingkat petani skala kecil yang hanya memiliki luas |',
 '<p>sawah 0,16 hektar dengan persentase tertinggi di Indonesia. Salah satu indikator tujuan |',
 '<p>pembangunan berkelanjutan pada sektor pertanian yaitu nilai produksi per hektar, 90% |',
 '<p>lahan pertanian di Jawa Barat, Jawa Timur, dan Nusa Tenggara Barat (NTB) dikategorikan |',
 '<p>la

In [91]:
doc = fitz.open("data/essay_3.pdf")
detected_fonts_styles = fonts(doc)
size_tag = font_tags(detected_fonts_styles[0],detected_fonts_styles[1])
tagged_list = headers_para(doc, size_tag)
tagged_list

['<p>Pengukuran dilapangan dilakukan untuk evapotranspirasi tanaman, perkolasi, air irigasi dan curah hujan. | Jumlah air yang diberikan di lahan yaitu hingga air tergenang di petak sawah setinggi 2 cm untuk perlakuan | sawah konvensional dan macak-macak untuk sawah SRI dan air akan dialirkan kembali ke lahan SRI ketika | tanah sawah sudah menunjukkan retak rambut. Pengukuran curah hujan dilakukan setiap hari dengan | menggunakan alat penakar hujan observatorium sederhana.  Pengukuran evapotranspirasi dan perkolasi | menggunakan susunan tiga buah  lysimeter  (Gambar 5). Tangki A berdasar terbuka dan ditanami padi | digunakan untuk mengukur jumlah kehilangan air tanaman oleh evapotranspirasi dan perkolasi (Et + P). | Tangki B berdasar terbuka dan tanpa ditanami padi, digunakan untuk mengukur evaporasi dan perkolasi (E + | P). Tangki C dengan dasar tertutup dan tanpa ditanami padi digunakan untuk mengukur evaporasi (E). | Pengukuran kehilangan air di dalam  lysimeter  menggunakan mistar.

# Layout-Parser

References: https://github.com/Layout-Parser/layout-parser/blob/master/examples/Deep%20Layout%20Parsing.ipynb

Pros: general solution for all kind of documents

Cons: big model size

In [112]:
#conda install -c conda-forge poppler
!pip install pdf2image

In [94]:
!pip install torch==1.8.1

     |████████████████████████████████| 804.1 MB 13 kB/s  eta 0:00:01     |████▉                           | 120.9 MB 3.7 MB/s eta 0:03:05     |██████████▍                     | 259.7 MB 4.0 MB/s eta 0:02:17     |██████████▋                     | 265.3 MB 3.7 MB/s eta 0:02:27     |██████████████▌                 | 363.5 MB 4.2 MB/s eta 0:01:44     |████████████████                | 401.1 MB 1.3 MB/s eta 0:05:11     |███████████████████▉            | 497.6 MB 2.1 MB/s eta 0:02:25     |█████████████████████▊          | 546.2 MB 2.8 MB/s eta 0:01:32     |████████████████████████████    | 700.9 MB 820 kB/s eta 0:02:06     |█████████████████████████████▎  | 734.8 MB 799 kB/s eta 0:01:27     |█████████████████████████████▌  | 742.3 MB 97 kB/s eta 0:10:34
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
ERROR: After October 2020 you may experience errors when installing or updating packages. Thi

In [96]:
!pip install -U layoutparser[ocr]

Requirement already up-to-date: layoutparser[ocr] in /home/louisowen6/miniconda3/envs/test_env/lib/python3.6/site-packages (0.2.0)
     |████████████████████████████████| 435 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 558 kB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 956 kB/s eta 0:00:01
     |████████████████████████████████| 146 kB 875 kB/s eta 0:00:01
     |████████████████████████████████| 198 kB 613 kB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 709 kB/s eta 0:00:01
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13946 sha256=1c4a5a10536ee6d64c1f1573e99752b31947a913e38d2e6e481b0440b2cd156d
  Stored in directory: /home/louisowen6/.cache/pip/wheels/c1/ca/d1/14e0dabe6f772bb5b45e006111268d943d4196fcb8798e3bf8
Successfully built pytesseract


In [ ]:
# sudo apt install tesseract-ocr-ind
# sudo apt install libtesseract-dev

In [3]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.8/index.html
     |████████████████████████████████| 5.1 MB 530 kB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 2.0 MB/s eta 0:00:01
  Using cached Cython-0.29.23-cp36-cp36m-manylinux1_x86_64.whl (2.0 MB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
     |████████████████████████████████| 3.9 MB 159 kB/s eta 0:00:01
     |████████████████████████████████| 129 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 1.0 MB/s eta 0:00:01
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=371759 sha256=893cf75b788ac140db175bb5b295d520fbecec569b0b0ac295c03f759434a2a5
  S

  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-py3-none-any.whl size=58541 sha256=7d296e19f4a6e881eee9fcb25a63a1e8f9fa4d9914f337eb5e811c73f810e2e3
  Stored in directory: /home/louisowen6/.cache/pip/wheels/c6/25/b5/1f76e3c22d316deb81d3837756cf497e86b817db223a4b49c6
Successfully built pycocotools fvcore


In [1]:
from pdf2image import convert_from_path
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

import re
import pandas as pd
import numpy as np

import layoutparser as lp
from layoutparser.elements import Rectangle

In [11]:
def pdf_to_text(pdf_path):
    # Convert PDF to Images
    images = convert_from_path(pdf_path)
    images = [np.asarray(images[i]) for i in range(len(images))]
    
    # Load the deep layout model from the layoutparser API 
    # For all the supported model, please check the Model 
    # Zoo Page: https://layout-parser.readthedocs.io/en/latest/notes/modelzoo.html

    model = lp.Detectron2LayoutModel(
                                     'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config', 
                                     extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                     label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

    # Initialize the tesseract ocr engine
    ocr_agent = lp.TesseractAgent(languages='ind')
    
    txt_list = []
    for image in images:
        text_blocks = extract_paragraphs(image,model,ocr_agent)

        if text_blocks is not None:
#             #Visualize the cleaned text blocks:
#             display(lp.draw_box(image, text_blocks,
#                         box_width=3, 
#                         show_element_id=True))


            for txt in text_blocks.get_texts():
                txt = re.sub('\n',' ',txt).strip()
#                 print(txt, end='\n---\n')
                txt_list.append(re.sub('\x0c','',txt))
    
    #Append splitted paragraph
    excluded_id = []
    new_txt_list = []
    for i in range(len(txt_list)-1):
        if txt_list[i+1][0] != txt_list[i+1][0].lower():
            if i not in excluded_id:
                new_txt_list.append(txt_list[i])
        else:
            if i not in excluded_id:
                new_txt_list.append(txt_list[i]+' '+txt_list[i+1])

                excluded_id.append(i+1)

    if (len(txt_list) - 1) not in excluded_id:
        new_txt_list.append(txt_list[-1])
    
    return new_txt_list

In [6]:
def extract_paragraphs(image,model,ocr_agent):
    # Detect the layout of the input image
    layout = model.detect(image)
    
    #Firstly we filter text region of specific type:
    text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
    figure_blocks = lp.Layout([b for b in layout if b.type=='Title'])
    
    #As there could be text region detected inside the figure region, we just drop them:
    text_blocks = lp.Layout([b for b in text_blocks \
                       if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
    
    # ----------------------
    # For 2 columns type of doc
    # #Finally sort the text regions and assign ids:

    # h, w = image.shape[:2]

    # left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

    # left_blocks = text_blocks.filter_by(left_interval, center=True)
    # left_blocks.sort(key = lambda b:b.coordinates[1])

    # right_blocks = [b for b in text_blocks if b not in left_blocks]
    # right_blocks.sort(key = lambda b:b.coordinates[1])

    # # And finally combine the two list and add the index
    # # according to the order
    # text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])
    # ----------------------
    
    if len(text_blocks)>0:
        # Indexing
        y1_loc_list = []
        for block in text_blocks:
            y1_loc_list.append(block.to_dict()['y_1'])

        id_list = (pd.Series(y1_loc_list).rank()-1).astype(int).to_list()

        text_blocks = lp.Layout([b.set(id = id_list[idx]) for idx, b in enumerate(text_blocks)])


        # Sorting
        text_blocks = lp.Layout([x for _,x in sorted(zip(id_list,text_blocks))])

        # Expanding
        y1_loc_list = []
        y2_loc_list = []
        for block in text_blocks:
            y1_loc_list.append(block.to_dict()['y_1'])
            y2_loc_list.append(block.to_dict()['y_2'])

        excluded_id = []
        expanded_text_blocks = []
        for i in range(len(y2_loc_list)-1):
            if (y1_loc_list[i]>=y1_loc_list[i+1]) and (y1_loc_list[i]<=y2_loc_list[i+1]):
                block = text_blocks[i]
                next_block = text_blocks[i+1]

                block.set(block=Rectangle(x_1=min(block.to_dict()['x_1'],next_block.to_dict()['x_1']),
                                          x_2=max(block.to_dict()['x_2'],next_block.to_dict()['x_2']),
                                          y_1=min(block.to_dict()['y_1'],next_block.to_dict()['y_1']),
                                          y_2=max(block.to_dict()['y_2'],next_block.to_dict()['y_2']))
                         , inplace=True)

                expanded_text_blocks.append(block)
                excluded_id.append(i+1)
            elif (y1_loc_list[i+1]>=y1_loc_list[i]) and (y1_loc_list[i+1]<=y2_loc_list[i]):
                block = text_blocks[i]
                next_block = text_blocks[i+1]

                block.set(block=Rectangle(x_1=min(block.to_dict()['x_1'],next_block.to_dict()['x_1']),
                                          x_2=max(block.to_dict()['x_2'],next_block.to_dict()['x_2']),
                                          y_1=min(block.to_dict()['y_1'],next_block.to_dict()['y_1']),
                                          y_2=max(block.to_dict()['y_2'],next_block.to_dict()['y_2']))
                         , inplace=True)

                expanded_text_blocks.append(block)
                excluded_id.append(i+1)
            else:
                if i not in excluded_id:
                    expanded_text_blocks.append(text_blocks[i])

        if (len(y2_loc_list) - 1) not in excluded_id:
            expanded_text_blocks.append(text_blocks[len(text_blocks)-1])

        expanded_text_blocks = lp.Layout(expanded_text_blocks)

        for block in expanded_text_blocks:
            # add padding in each image segment can help improve robustness 
            segment_image = (block
                               .pad(left=5, right=5, top=5, bottom=5)
                               .crop_image(image))


            text = ocr_agent.detect(segment_image)
            block.set(text=text, inplace=True)

        return expanded_text_blocks
    else:
        return None

In [13]:
essay_1_doc = pdf_to_text('data/essay_1.pdf')
essay_1_doc

['Musim kemarau di Indonesia menyebabkan angka produksi padi menurun, penurunan sebesar 4,6 juta ton terjadi di tahun 2019 (1). Gambar 1 menunjukan luas fase persiapan lahan terjadi penurunan pada bulan April — Agustus yang sesuai dengan musim kemarau di Indonesia. Provinsi yang memiliki luas fase persiapan terbesar diantaranya Jawa Barat,  Jawa Tengah, Jawa Timur, Sulawesi Selatan, dan Sumatera Selatan.',
 'Selain itu, provinsi tersebut memiliki tingkat petani skala kecil yang hanya memiliki luas sawah 0,16 hektar dengan persentase tertinggi di Indonesia. Salah satu indikator tujuan pembangunan berkelanjutan pada sektor pertanian yaitu nilai produksi per hektar, 90Y6 lahan pertanian di Jawa Barat, Jawa Timur, dan Nusa Tenggara Barat (NTB) dikategorikan lahan pertanian tidak berkelanjutan (21. Hal ini disebabkan karena tidak ada sumber air saat musim kemarau, bantuan dari pemerintah berupa pompa air tetapi sumber listrik di lokasi belum memadai. Mahalnya harga dan jauhnya akses untuk m

In [12]:
essay_1_similar_doc = pdf_to_text('data/essay_1_similar.pdf')
essay_1_similar_doc

['Oleh: Reiner Nathaniel J',
 'Musim kemarau di Indonesia menyebabkan angka produksi padi menurun, penurunar sebesar 4,6 juta ton terjadi di tahun 2019 (1). Gambar 1 menunjukan luas fase persiapar lahan terjadi penurunan pada bulan April — Agustus yang sesuai dengan musim kemarau d Indonesia. Provinsi yang memiliki luas fase persiapan terbesar diantaranya Jawa Barat  Jawa Tengah, Jawa Timur, Sulawesi Selatan, dan Sumatera Selatan.',
 'Gambar 1. Luas fase persiapan lahan 2018 — 2019 (Dimodifikasi dari BPS, 2019)',
 'Selain itu, provinsi tersebut memiliki tingkat petani skala kecil yang hanya memiliki luas sawah 0,16 hektar dengan persentase tertinggi di Indonesia. Salah satu indikator tujuan pembangunan berkelanjutan pada sektor pertanian yaitu nilai produksi per hektar, 90” lahan pertanian di Jawa Barat, Jawa Timur, dan Nusa Tenggara Barat (NTB) dikategorikan lahan pertanian tidak berkelanjutan (2). Hal ini disebabkan karena tidak ada sumber air saat musim kemarau, bantuan dari pemerin

In [14]:
essay_2_doc = pdf_to_text('data/essay_2.pdf')
essay_2_doc

['Pemerintah Indonesia ingin menurunkan emisi sebesar 29x pada tahun 2030 sehubungan dengan partisipasi Indonesia dalam Perjanjian Paris 2015 (1). Tindakan iklim menjadi fokus global, urgensi untuk mengatasi semua emisi karbon dioksida dari banyak negara di seluruh dunia. Perhatian pemerintah pertama-tama difokuskan pada sektor ketenagalistrikan untuk memanfaatkan energi terbarukan untuk mengurangi emisi global. Dengan meningkatkan energi terbarukan, negara dapat secara tajam mengurangi satu sumber utama terkait emisi karbon dioksida. Indonesia menetapkan target energi terbarukan dalam bauran energi sebesar 23X pada tahun 2025, pada O3 tahun 2020 persentasenya hanya 10,94 (2,31. Kesenjangan untuk mencapai tujuan sekitar 14,196, itu harus melipatgandakan usaha empat kali dari bisnis seperti biasa (31.',
 'Menurut konteks global energi trilema, biasanya digunakan untuk menyeimbangkan keamanan energi, ekuitas energi, dan keberlanjutan energi. Indonesia membutuhkan energi terbarukan untuk 

In [15]:
essay_2_similar_doc = pdf_to_text('data/essay_2_similar.pdf')
essay_2_similar_doc

['Transisi energi akan menghadapi banyak tantangan, kebijakan menjadi aspek kunci untuk mempercepat transisi energi. Strategi pemerintah saat ini adalah dengan menerapkan cot-firing, substitusi batubara menjadi biomassa pada tahap awal dengan sensitivitas pallet / woodchip menjadi batubara sekitar 1-54. Strategi ini akan merugikan sumber daya lain, kapasitas yang besar dapat memenuhi permintaan dan sumber daya energi terbarukan lainnya akan terus berkembang atau beralih ke prioritas yang berbeda. . Pemerintah Indonesia ingin menurunkan emisi sebesar 2996 pada tahun 2030 sehubungan dengan partisipasi Indonesia dalam Perjanjian Paris 2015 (1). Tindakan iklim menjadi fokus global, urgensi untuk mengatasi semua emisi karbon dioksida dari banyak negara di seluruh dunia. Perhatian pemerintah pertama-tama difokuskan pada sektor ketenagalistrikan untuk memanfaatkan energi terbarukan untuk mengurangi emisi global. Dengan meningkatkan energi terbarukan, negara dapat secara tajam mengurangi satu 

In [16]:
essay_3_doc = pdf_to_text('data/essay_3.pdf')
essay_3_doc

['Pengukuran dilapangan dilakukan untuk evapotranspirasi tanaman, perkolasi, air irigasi dan curah hujan. Jumlah air yang diberikan di lahan yaitu hingga air tergenang di petak sawah setinggi 2 cm untuk perlakuan sawah konvensional dan macak-macak untuk sawah SRI dan air akan dialirkan kembali ke lahan SRI ketika tanah sawah sudah menunjukkan retak rambut. Pengukuran curah hujan dilakukan setiap hari dengan menggunakan alat penakar hujan observatorium sederhana. Pengukuran evapotranspirasi dan perkolasi menggunakan susunan tiga buah Iysimeter (Gambar 5). Tangki A berdasar terbuka dan ditanami padi digunakan untuk mengukur jumlah kehilangan air tanaman oleh evapotranspirasi dan perkolasi (Et # P). Tangki B berdasar terbuka dan tanpa ditanami padi, digunakan untuk mengukur evaporasi dan perkolasi (E P). Tangki C dengan dasar tertutup dan tanpa ditanami padi digunakan untuk mengukur evaporasi (E). Pengukuran kehilangan air di dalam Iysimeter menggunakan mistar. Kehilangan air setiap harin